## CNN Attempt for ISIC 2024 - Skin Cancer Detection with 3D-TBP

Present issue is the massive imbalance in the dataset of positives compared to negatives. To address this issue, I will utilize a custom sampler object called `ImbalancedDatasetSampler` when initiating DataLoader objects.

Another step to increase the effectiveness of CNN on binary classification of skin cancer on the image dataset is using an ensemble of CNN's which is commonly known to increase overall performances.

## Import Libraries

In [10]:
"""
Import libraries
"""
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.nn as nn
import torchvision
from io import BytesIO
import h5py
import random
import os

## Training Configuration

In [8]:
CONFIG = {
    "seed": 42,
    "epochs": 20,
    "img_size": 336,
    "train_batch_size": 32,
    "valid_batch_size": 64,
    "learning_rate": 1e-5,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [12]:
"""
Set seed and 
"""

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#torch.cuda.set_per_process_memory_fraction(0.5, 0)


SEED = 111
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

ROOT_DIR = "/kaggle/input/isic-2024-challenge"
TRAIN_CSV = f"{ROOT_DIR}/train-metadata.csv"
TRAIN_HDF = f"{ROOT_DIR}/train-image.hdf5"
TEST_CSV = f'{ROOT_DIR}/test-metadata.csv'
TEST_HDF = f'{ROOT_DIR}/test-image.hdf5'
SAMPLE = f'{ROOT_DIR}/sample_submission.csv'

## Dataset Class

In [ ]:
class ISICDataset_for_Train(Dataset):
    def __init__(self, df, file_hdf, transforms=None):
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.df_positive = df[df["target"] == 1].reset_index()
        self.df_negative = df[df["target"] == 0].reset_index()
        self.isic_ids_positive = self.df_positive['isic_id'].values
        self.isic_ids_negative = self.df_negative['isic_id'].values
        self.targets_positive = self.df_positive['target'].values
        self.targets_negative = self.df_negative['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df_positive) * 2
    
    def __getitem__(self, index):
        if random.random() >= 0.5:
            df = self.df_positive
            isic_ids = self.isic_ids_positive
            targets = self.targets_positive
        else:
            df = self.df_negative
            isic_ids = self.isic_ids_negative
            targets = self.targets_negative
        index = index % df.shape[0]
        
        isic_id = isic_ids[index]
        img = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        target = targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }
    
class ISICDataset(Dataset):
    def __init__(self, df, file_hdf, transforms=None):
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.df = df
        self.isic_ids = df['isic_id'].values
        self.targets = df['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        img = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }